# TF-IDF 구하기
- 벡터화하기 전에 데이터 전처리
   - document별 형태소 분석
   - 미등록단어 처리
   - 명사만 추출
   - 불용어(stopwords) 처리
- document별 단어로 tokenize되어 있는 것을 다시 합쳐서 각각 하나의 document로 만들기
- 벡터화(vectorize) 하기
- tf-idf 값 구하기

In [1]:
import pandas as pd

In [2]:
# CSV 데이터 불러오기
df = pd.read_csv("../../data/naver_blog_clova.csv", index_col=0)

In [3]:
# 데이터 확인
df.head()

,title,publish_datetime,text,url
0,애기들이 좋아하는 ai스피커 네이버클로바,2019-03-20 23:01:00,애기들이 좋아하는 ai스피커 네이버클로바\n작년부터 tv에 #인공지능 스피커가 \n...,https://blog.naver.com/sky121400?Redirect=Log&...
1,네이버 클로바 스피커 1년 사용 후기,2019-03-31 19:35:00,네이버 클로바 스피커 1년 사용 후기\n네이버 클로바 1년 사용 후기를 몇 자 적어...,https://blog.naver.com/danjuck?Redirect=Log&lo...
2,네이버 클로바 사용법 맘스다이어리와 함께 육아정보를,2018-12-14 12:10:00,어제는 눈이 갑자기 많이 내려서 1주일에 한번 가는 문센을 가야하나 말아야하나고민이...,https://blog.naver.com/ralrapo?Redirect=Log&lo...
3,네이버 클로바. 나의 두 번째 Ai 스피커,2018-12-02 14:04:00,안녕하세요. 제르디난 입니다.\n올해 초 카카오 미니를 사용해보고 약간의 실망스러움...,https://blog.naver.com/yarosiku?Redirect=Log&l...
4,네이버 클로바 스피커 브라운 미니언 : 오히려 부모님 선물로 추천드려요!,2018-09-09 18:18:00,네이버 클로바 스피커 브라운 미니언 : 오히려 부모님 선물로 추천드려요!\n요새 정...,https://blog.naver.com/ukb0914?Redirect=Log&lo...


In [4]:
# 블로그 본문 확인
df.text[:5]

0    애기들이 좋아하는 ai스피커 네이버클로바\n작년부터 tv에 #인공지능 스피커가 \n...
1    네이버 클로바 스피커 1년 사용 후기\n네이버 클로바 1년 사용 후기를 몇 자 적어...
2    어제는 눈이 갑자기 많이 내려서 1주일에 한번 가는 문센을 가야하나 말아야하나고민이...
3    안녕하세요. 제르디난 입니다.\n올해 초 카카오 미니를 사용해보고 약간의 실망스러움...
4    네이버 클로바 스피커 브라운 미니언 : 오히려 부모님 선물로 추천드려요!\n요새 정...
Name: text, dtype: object

In [5]:
# 블로그 본문 컬럼이 Not null인 총 개수 확인
len(df[df.text.notnull()])

1052

In [6]:
# 블로그 본문 컬럼이 Not null인 row를 제외하기
df = df[df.text.notnull()].copy()

In [7]:
# 블로그 본문에 있는 개행문자(\n, \r)를 제거
df.text = df.text.map(lambda x:x.replace("\n", " ").replace("\r", " "))

<br/><br/>
### POS tagging하기

#### 1.  Komoran 사용

In [8]:
from konlpy.tag import Komoran

In [15]:
komoran = Komoran(userdic='./unregistered.txt')

In [16]:
pos_tagging_list = list(map(komoran.pos, df.text))

In [17]:
import re

# 명사와 외래어만 뽑아내기위한 정규표현식
p = re.compile('NN.*|SL')

In [18]:
docs = []
for doc in pos_tagging_list:
    want_words = []
    for word, pos in doc:
        if p.match(pos):
            want_words.append(word)
    docs.append(want_words)

In [19]:
docs[:2]

[['애기',
  'ai',
  '스피커',
  '네이버',
  '클로바',
  '작년',
  'tv',
  '인공지능',
  '스피커',
  '출현',
  '시기상조',
  '생각',
  '애플',
  'siri',
  '구글',
  '기능',
  '인공',
  '스피커',
  '라고',
  '어느날',
  '형',
  '네이버',
  '클로바',
  '생각',
  '말',
  '애',
  '구매',
  '권유',
  '애',
  '말',
  '귀',
  '!!',
  '집',
  'ai',
  '스피커',
  '폭풍',
  '검색',
  '아마존',
  '알렉사',
  '해외',
  '우리나라',
  '통신',
  '스피커',
  'sk',
  'nugu',
  'kt',
  '기가',
  '지니',
  'lg',
  '구글',
  '구글',
  '홈',
  '네이버',
  '클로바',
  '경쟁',
  '만사',
  '분',
  '통신',
  '스피커',
  'iot',
  '구축',
  '고민',
  '분',
  '구글',
  '홈',
  '것',
  '중',
  '네이버',
  '클로바',
  '통신',
  '강제',
  'iot',
  '구축',
  '수',
  '구글',
  '홈',
  '확장',
  '것',
  '것',
  '우리나라',
  '최적화',
  '것',
  '캐릭터',
  '모양',
  '장점',
  '미니언즈',
  '샐리',
  '브라운',
  '이름',
  '스피커',
  '존재',
  '소형',
  '미니',
  '스피커',
  '배터리',
  '향상',
  '존재',
  'sk',
  'nugu',
  'vs',
  '네이버',
  '클로바',
  '처음',
  '클로바',
  '구매',
  'ai',
  '스피커',
  '집',
  '활용',
  '생각',
  'sk',
  'nugu',
  '캔들',
  '을',
  '검색',
  '집',
  'iptv',
  'sk',
  'nugu',
  '사용',
  'tv',
  

<br/><br/>
### stopword 제거

In [20]:
stopwords = [line.strip() for line in open('./stopwordsKor.txt', encoding='utf-8')]

In [23]:
final_docs = []
for doc in docs:
    unique_NN_words = set(doc)
    final_NN_words = doc
    
    for word in unique_NN_words:
        if word in stopwords:
            while word in final_NN_words: final_NN_words.remove(word)
    final_docs.append(final_NN_words)

<br/><br/>
### document별 단어들을 각각의 document로 바꾸기

In [26]:
documents = []
for doc_words in final_docs:
    document = " ".join(doc_words)
    documents.append(document)

In [27]:
documents[0]

'애기 ai 스피커 네이버 클로바 작년 tv 인공지능 스피커 출현 시기상조 애플 siri 구글 기능 인공 스피커 어느날 네이버 클로바 애 구매 권유 애 귀 ai 스피커 폭풍 검색 아마존 알렉사 해외 우리나라 통신 스피커 sk nugu kt 기가 지니 lg 구글 구글 홈 네이버 클로바 경쟁 만사 통신 스피커 iot 구축 고민 구글 홈 네이버 클로바 통신 강제 iot 구축 구글 홈 확장 우리나라 최적화 캐릭터 모양 장점 미니언즈 샐리 브라운 이름 스피커 존재 소형 미니 스피커 배터리 향상 존재 sk nugu vs 네이버 클로바 처음 클로바 구매 ai 스피커 활용 sk nugu 캔들 검색 iptv sk nugu 사용 tv 제어 와이프 캔들 활용도 원래 목적 애 차이 클로바 장형 배터리 배터리 클로바 휴대 고정 사용 전기 단점 클로바 구입 결정 구입 애 모델 예상외 곰 곰 구입 뒤 택배 세팅 설명 휴대폰 설치 세팅 기기 와이파이 검색 비번 스피커 이름 끝 스피커 디바 이스 호출 스피커 이름 우리집 곰 돌이 브라운 스피커 이름 샐리 기기 음성 스피커 목소리 기호 선택 애 이름 사용법 리다 손 모습 고민 신기하 게도 어른 반응 애 반응 음역 이해 애 발음 스피커 애기 필요 장난감 스피커 성능 애 관심 스피커 조금 처음 친구 하루 종일 샐리 노이로제 나중 스피커 이틀 아무도 아침 스피커 노래 날씨 어른 장난감 네이버 클로바 휴대 장거리 이동 여행 애 노래 이야기 충전 c 타입 외장 배터리 사용 차량 충전 사용 와이파이 휴대폰 핫 스판 이용 설정 처음 클로바 단점 장소 와이파이 설정 캠핑장 사용 기기 와이파이 핫스팟 야외 사용 와이파이 기억 세팅 설정 와이파이 세팅 단점 화나 스피커 듯'

<br/><br/>
### 벡터화 하기

In [28]:
from sklearn.feature_extraction.text import CountVectorizer     # 벡터 수 카운트 할 때

In [32]:
cv=CountVectorizer(max_df=0.85, max_features=10000)     
# 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW(Bag Of Words) 인코딩한 벡터를 만든다.
# max_df=0.85란 말은 documents들에서 85%이상 나타나는 토큰(단어)를 무시하라는 것, 
# max_df=25란 말은 documents들에서 25번 이상 나타나는 토큰(단어)를 무시하라는 것,
# maximum 단어의 개수를 1만개로 정함 - 빈도수가 높은 단어 순으로 1만개를 자름(메모리 에러 안나려고)
# min_df=1, ngram_range=(1,1) 등은 default 값임  

word_count_vector=cv.fit_transform(documents)      # vectorize된 word_count_vector

In [35]:
len(cv.vocabulary_)      # 위에서 max_features를 10000으로 주었지만 단어 수가 그 이하였음

8955

In [38]:
cv.vocabulary_

{'애기': 5377,
 'ai': 146,
 '스피커': 4936,
 '작년': 6694,
 'tv': 1176,
 '인공지능': 6474,
 '출현': 7724,
 '시기상조': 4969,
 '애플': 5399,
 'siri': 1061,
 '구글': 1904,
 '기능': 2116,
 '인공': 6472,
 '어느날': 5464,
 '구매': 1915,
 '권유': 1994,
 '폭풍': 8317,
 '검색': 1568,
 '아마존': 5235,
 '알렉사': 5348,
 '해외': 8641,
 '우리나라': 5997,
 '통신': 8102,
 'sk': 1062,
 'nugu': 877,
 'kt': 703,
 '기가': 2104,
 '지니': 7325,
 'lg': 722,
 '경쟁': 1648,
 '만사': 3199,
 'iot': 648,
 '구축': 1957,
 '고민': 1717,
 '강제': 1471,
 '확장': 8838,
 '최적화': 7667,
 '캐릭터': 7820,
 '모양': 3408,
 '장점': 6780,
 '미니언즈': 3562,
 '샐리': 4445,
 '브라운': 4162,
 '이름': 6336,
 '존재': 7151,
 '소형': 4726,
 '미니': 3560,
 '배터리': 3781,
 '향상': 8675,
 'vs': 1229,
 '처음': 7549,
 '활용': 8860,
 '캔들': 7824,
 'iptv': 651,
 '사용': 4311,
 '제어': 7073,
 '와이프': 5882,
 '활용도': 8861,
 '원래': 6061,
 '목적': 3431,
 '차이': 7483,
 '장형': 6788,
 '휴대': 8923,
 '고정': 1753,
 '전기': 6864,
 '단점': 2553,
 '구입': 1948,
 '결정': 1612,
 '모델': 3386,
 '예상외': 5767,
 '택배': 8043,
 '세팅': 4657,
 '설명': 4570,
 '휴대폰': 8924,
 '설치': 4576,
 '기기

<br/><br/>
### TF-IDF 적용
- TF * IDF는 특정 문서 내에서 단어 빈도가 높을수록, 전체 문서들에는 그 단어를 포함한 문서가 적을수록 TF * IDF값이 높아지는 특징이 있다.
- 이러한 특징을 이용해서 모든 문서에 나타나는 흔한 단어들을 걸러내며, 특정 단어가 가지는 중요도를 측정하는 데 사용된다.
- 한마디로 TF * IDF 값은 특정 단어가 가지는 중요도!
- TF(Term Frequency): 단어 빈도
   - 해당 문서에서 단어가 나타나는 빈도수
   - 문서의 길이가 길면 해당 단어의 실제 중요도와는 상관없이 단어의 빈도수는 증가될 확률이 높다.
   - 위의 문제를 해결하기 위해 다음과 같이 표준화 -> (문서에서 단어가 나타나는 빈도수) / (모든 단어가 나타나는 빈도수)
- IDF(Inverse Document Frequency): 역문헌 빈도
   - 해당 단어의 일반적인 중요도를 나타내는 값
   - log( 전체 문서의 수 / 해당 단어가 포함된 문서들의 수 )

In [39]:
from sklearn.feature_extraction.text import TfidfTransformer    # Tf * idf 구할 때

In [52]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)        # Tf-idf 가중치를 적용할 수 있도록 변환시켜줌
tf_idf_matrix = tfidf_transformer.fit_transform(word_count_vector)

dense = tf_idf_matrix.todense()

for i in range(len(dense)):
    doc = dense[i].tolist()[0]           # dense[i].tolist()는 2차원 list, 예를들면 shape: (1,814)
    phrase_scores = [pair for pair in zip(range(0, len(doc)), doc) if pair[1] > 0]      # 0의 값이 아닌것만 모아서 만듦


    sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1], reverse=True) # sorted(phrase_scores, key=lambda t: t[1] * -1)라고 해도 됨
    for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:20]:
        print('{0: <20} {1}'.format(phrase, score))      # 단어와 단어의 tf-idf 값을 출력
    print()
    ##### 이 부분은 테스트를 위해서 #####
    if i == 3:     
        break
    ###############################

스피커                  0.3098984343915409
nugu                 0.2645554291179715
sk                   0.22043403361752695
구글                   0.21233337232562116
세팅                   0.2099266307961928
와이파이                 0.19848448818510178
통신                   0.18329517379444307
이름                   0.16356646568656333
애기                   0.14326284214456464
배터리                  0.14069765591348146
단점                   0.14002783767846855
캔들                   0.13827363937277373
존재                   0.12745048394029582
어른                   0.1260293750130323
우리나라                 0.1260293750130323
장난감                  0.12340983469825578
구축                   0.11993498417918466
사용                   0.10965867046326969
구입                   0.10779796862956097
처음                   0.09654512049625422

동요                   0.2666462648454195
배터리                  0.22693249566084311
점수                   0.2268448274777323
뒤통수                  0.21632371392852515
공짜                   0